In [2]:
import sys
import os
sys.path.append(os.path.abspath('../src'))

In [ ]:
from data_processor import DataProcessor
from data_processor import CycleProcessor
from feature_engineering import FeatureEngineering
from model.coarse_feature_selection.cfs import random_forest_regressor
from model.fine_feature_selection.ffs import fine_feature_selection
import pandas as pd
import numpy as np
from model.lstm_model.enhanced_lstm import lstm_model
file_path = "../data/raw/data_factory_1.xlsx"
dp = DataProcessor(file_path)
dp.change_pivot('timestamp','param_name','value')
dp.drop_NA_with_feature(features=['FeedFlow','FeedTemperature'])
dp.rename_column_to_timestamp('timestamp')
dp.rename_column_to_permeatepressure('Permeate Pressure')
dp.list_columns()
cp = CycleProcessor(column_name='FeedFlow', df = dp.df, threshold=10)
cp.identify_cycles()
cp.assign_cycle_features()
file_path = '../data/cycle_processing_data/factory1.csv'
cp.export_files(file_path)
fe = FeatureEngineering(dp)
fe.generate_cross_features(drop_features=['Recovery', 'PermeateFlow', 'PermeateConductivity', 'Permeate Pressure'])
fe.lag_engineer()
dp.df = fe.df
features =dp.df.columns.tolist()
fs = [f for f in features if f != ''and 'timestamp' not in f and 'ConcentratePressure' not in f  and 'PermeatePressure' not in f and 'PermeateConductivity' not in f and f not in ['PermeateFlow'] ]
# #and'Permeate' not in f Permeate Pressure
# print(fs)


top_k_features = random_forest_regressor(dp, 'PermeateFlow', fs, plant_name='plant1')

top_k_features = pd.read_csv("../data/temp_data/top_k_features_plant1_PermeateFlow.csv")
top_k_features = top_k_features.iloc[:,1].tolist()
target = ['PermeateFlow']
s_features = fine_feature_selection(dp, top_k_features, target)
lstm_model(dp, top_k_features, target)